# Day 11

## Part 1

- The monkeys operate based on how worried I am about each item
- Each monkey has the following attributes:
    - Starting items: a list of the worry level per item
    - Operation: how my worry level changes as the monkey inspects the item
    - Test: How the monkey decides whether/where to throw the item next
- After the operation but before the test the worry level is divided by 3
- Monkeys go in order of their number. When all the monkeys have gone, that is a round.
- Items are added to the end of a monkey slist of items.

`Chasing all of the monkeys at once is impossible; you're going to have to focus on the two most active monkeys if you want any hope of getting your stuff back. Count the total number of times each monkey inspects items over 20 rounds:`

`Figure out which monkeys to chase by counting how many items they inspect over 20 rounds. What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans?`

In [75]:
from utils import parse_from_file, ParseConfig

file = 'day_11.txt'
parser = ParseConfig('\n\n', str)

initial_state = parse_from_file(file, parser)

print(initial_state[0])

Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3


In [76]:
# looks like we're in for some parser shenannigans!
parser = ParseConfig('\n\n', ParseConfig('\n', str))

initial_state = parse_from_file(file, parser)

print(initial_state[0])

['Monkey 0:', '  Starting items: 79, 98', '  Operation: new = old * 19', '  Test: divisible by 23', '    If true: throw to monkey 2', '    If false: throw to monkey 3']


In [77]:
parser = ParseConfig('\n\n', ParseConfig('\n', [
    ParseConfig(':', [ParseConfig(' ', [None, int]), None]),
    str, str, str, str, str
]))

initial_state = parse_from_file(file, parser)

print(initial_state[0])

[[[0]], '  Starting items: 79, 98', '  Operation: new = old * 19', '  Test: divisible by 23', '    If true: throw to monkey 2', '    If false: throw to monkey 3']


In [78]:
parser = ParseConfig('\n\n', ParseConfig('\n', [
    ParseConfig(':', [ParseConfig(' ', [None, int]), None]),  # Monkey index
    ParseConfig('Starting items:', [None, ParseConfig(', ', int)]),
    str, str, str, str
]))

initial_state = parse_from_file(file, parser)

print(initial_state[0])

[[[0]], [[79, 98]], '  Operation: new = old * 19', '  Test: divisible by 23', '    If true: throw to monkey 2', '    If false: throw to monkey 3']


In [79]:
parser = ParseConfig('\n\n', ParseConfig('\n', [
    ParseConfig(':', [ParseConfig(' ', [None, int]), None]),  # Monkey index
    ParseConfig('Starting items:', [None, ParseConfig(', ', int)]),
    ParseConfig('Operation: new = old ', [None, ParseConfig(' ', str)]),
    str, str, str
]))

initial_state = parse_from_file(file, parser)

print(initial_state[0])

[[[0]], [[79, 98]], [['*', '19']], '  Test: divisible by 23', '    If true: throw to monkey 2', '    If false: throw to monkey 3']


In [80]:
parser = ParseConfig('\n\n', ParseConfig('\n', [
    ParseConfig(':', [ParseConfig(' ', [None, int]), None]),  # Monkey index
    ParseConfig('Starting items:', [None, ParseConfig(', ', int)]),
    ParseConfig('Operation: new = old ', [None, ParseConfig(' ', str)]),
    ParseConfig('Test: divisible by', [None, int]),
    str, str
]))

initial_state = parse_from_file(file, parser)

print(initial_state[0])

[[[0]], [[79, 98]], [['*', '19']], [23], '    If true: throw to monkey 2', '    If false: throw to monkey 3']


In [81]:
parser = ParseConfig('\n\n', ParseConfig('\n', [
    ParseConfig(':', [ParseConfig(' ', [None, int]), None]),  # Monkey index
    ParseConfig('Starting items:', [None, ParseConfig(', ', int)]),
    ParseConfig('Operation: new = old ', [None, ParseConfig(' ', str)]),
    ParseConfig('Test: divisible by', [None, int]),
    ParseConfig('If true: throw to monkey', [None, int]),
    ParseConfig('If false: throw to monkey', [None, int]),
]))

initial_state = parse_from_file(file, parser)

print(initial_state[0])

[[[0]], [[79, 98]], [['*', '19']], [23], [2], [3]]


In [82]:
class Monkey:
    """A container for all the business attributed to one monkey"""
    def __init__(self, parser_dump: 'list[list]'):
        id, items, operation, test, on_true, on_false = parser_dump
        self.id = id[0][0]
        self.items = items[0]
        self.operation = self._parse_operation(*operation[0])
        self.test = self._parse_test(test[0])
        self.on_true = on_true[0]
        self.on_false = on_false[0]

        self.divisor = test[0]
        self.inspection_count = 0
    
    def _parse_operation(
            self, operator: str, operand: str) -> callable:
        """
        returns the operation as a function by parsing the values specified in
        the input.
        """
        def operation(value: int) -> int:
            if operand == 'old':
                op2 = value
            else:
                op2 = int(operand)
            if operator == '+':
                return value + op2
            elif operator == '*':
                return value * op2
            else:
                raise ValueError(f'operator not recognised: {operator}')
        
        return operation

    def _parse_test(self, divisor: int) -> callable:
        """
        returns a function that returns the index of the monkey an item should
        go to next
        """
        def divisor_check(value: int) -> int:
            return self.on_true if value % divisor == 0 else self.on_false
        
        return divisor_check
    
    def inspect(self) -> 'list[tuple[int, int]]':
        """
        operates on a monkey's items in turn and updates the inspection count

        all values are cast to integer
        """
        outcomes = list()
        for item in self.items:
            new_value = int(self.operation(item) / 3)
            new_index = self.test(new_value)
            outcomes.append((new_index, new_value))
            self.inspection_count += 1

        self.items = list()

        return outcomes

    def __str__(self) -> str:
        string = f'<{self.__class__}: '
        attributes = []
        for attribute in dir(self):
            if not attribute.startswith('_'):
                attributes.append(f'{attribute}={getattr(self, attribute)}')
        string = string + ', '.join(attributes) + '>'
        return string

In [83]:
monkeys = [Monkey(state) for state in initial_state]

print(monkeys[0])

for round in range(1, 20 + 1):
    for monkey in monkeys:
        allocations = monkey.inspect()
        for monkey_index, item in allocations:
            monkeys[monkey_index].items.append(item)

<<class '__main__.Monkey'>: divisor=23, id=0, inspect=<bound method Monkey.inspect of <__main__.Monkey object at 0x000002048FA4B910>>, inspection_count=0, items=[79, 98], on_false=3, on_true=2, operation=<function Monkey._parse_operation.<locals>.operation at 0x0000020490436480>, test=<function Monkey._parse_test.<locals>.divisor_check at 0x00000204904363E0>>


In [84]:
top_monkey = max([monkey.inspection_count for monkey in monkeys])

next_top_monkey = max([
    monkey.inspection_count for monkey in monkeys
    if monkey.inspection_count != top_monkey
])

print(
    f'the monkey business after 20 rounds is: {top_monkey * next_top_monkey}!')

the monkey business after 20 rounds is: 10605!


## Part 2

- worry level is no longer divided by 3 after inspection
- simulate for 10000 rounds!

`Starting again from the initial state in your puzzle input, what is the level of monkey business after 10000 rounds?`

In [85]:
# I had a rather hefty hint for this part
LOWEST_COMMON_MULTIPLE = 1
for monkey in monkeys:
    LOWEST_COMMON_MULTIPLE *= monkey.divisor

print(f'the most I need to bother worrying is: {LOWEST_COMMON_MULTIPLE}!')

the most I need to bother worrying is: 96577!


In [86]:
class CarelessMonkey(Monkey):
    """
    This monkey is just like a normal monkey but makes you so worried that
    during inspection the total is not divided by 3
    """
    def inspect(self) -> 'list[tuple[int, int]]':
        """
        operates on a monkey's items in turn and updates the inspection count
        """
        outcomes = []
        for item in self.items:
            new_value = self.operation(item) % LOWEST_COMMON_MULTIPLE
            new_index = self.test(new_value)
            outcomes.append((new_index, new_value))
            self.inspection_count += 1

        self.items = []

        return outcomes

In [87]:
initial_state = parse_from_file(file, parser)

careless_monkeys = [CarelessMonkey(state) for state in initial_state]

for round in range(1, 10000 + 1):
    for monkey in careless_monkeys:
        allocations = monkey.inspect()
        for monkey_index, item in allocations:
            careless_monkeys[monkey_index].items.append(item)
    print(f'round: {round} {sum([len(monkey.items) for monkey in careless_monkeys])}', end='\r')
print()

round: 10000 10


In [88]:
top_careless_monkey = max([
    monkey.inspection_count for monkey in careless_monkeys])

next_top_careless_monkey = max([
    monkey.inspection_count for monkey in careless_monkeys
    if monkey.inspection_count != top_careless_monkey
])

print(
    'the monkey business after 10,000 rounds is: '
    f'{top_careless_monkey * next_top_careless_monkey}!')

the monkey business after 10,000 rounds is: 2713310158!
